In [ ]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

In [ ]:
# Basic Libraries
import os
import re
from time import sleep
import pandas as pd
# pd.set_option('max_colwidth', None)

# Visualization
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import plotly
from plotly import tools
import plotly.express as px
from jupyter_dash import JupyterDash
plotly.offline.init_notebook_mode() # To embed plots in the output cell of the notebook

import IPython
from IPython.display import HTML, display, Markdown, IFrame

# Xena
import smart_open

# External Code
from utils import *

In [ ]:
%%appyter hide_code_exec
{% do SectionField(
    name = 'Xena_Section',
    title = 'Gene Expression Data from Xena',
    subtitle = '',
    img = 'xena.png'
)%}

{% do SectionField(
    name = 'Subgroup_Section',
    title = 'Select the Two Subgroups to Generate Differential Gene Expression Signature',
    subtitle = 'Click to choose your subgroups. Shift-click to select more than one category for a subgroup.',
    img = 'analysis.png'
)%}

{% do SectionField(
    name = 'GS_Libraries_Section',
    title = 'Gene Set Libraries',
    subtitle = 'Click to select a library to run blitzGSEA on.',
    img = 'analysis.png'
)%}

{% do SectionField(
    name = 'Visualization_Section',
    title = 'Advanced Parameters for PCA/t-SNE',
    subtitle = '',
    img = 'analysis.png',
    collapse = 'hide'
)%}

{% do SectionField(
    name = 'DEG_Section',
    title = 'Advanced Parameters for Generating Differential Gene Expression Signature',
    subtitle = '',
    img = 'analysis.png',
    collapse = 'hide'
)%}

{% do SectionField(
    name = 'blitzGSEA_Section',
    title = 'Advanced Parameters for blitzGSEA',
    subtitle = '',
    img = 'analysis.png',
    collapse = 'hide'
)%}

In [ ]:
%%appyter code_exec
{% set DE_params = PMSGStringField(
    name = 'DE_params', 
    label = 'Differential expression parameters', 
    default = '',
    section = 'Xena_Section'
) %}

In [ ]:
%%appyter code_exec
{% set case_group = CategoryMultiChoiceField(
    name = 'category_case',
    urlhashkey = 'category',
    label = 'Subgroup 1',
    description = 'Select category to be in the 1st subgroup',
    choices = [''],
    default = [''],
    section = 'Subgroup_Section'
)%}

{% set control_group = CategoryMultiChoiceField(
    name = 'category_control',
    urlhashkey = 'category',
    label = 'Subgroup 2',
    description = 'Select category to be in the 2nd subgroup',
    choices = [''],
    default = [''],
    section = 'Subgroup_Section'
)%}

In [ ]:
%%appyter code_exec
{% set gs_library = ChoiceField(
    name = 'gs_library',
    label = 'Gene Set Libraries',
    description = 'Select a gene set library',
    choices = {'MSigDB_Hallmark': 'h.all.v7.5.1.symbols.gmt', 'MSigDB_C1_Positional': 'c1.all.v7.5.1.symbols.gmt', 'MSigDB_C2_Chemical_Genetic_Perturbations': 'c2.cgp.v7.5.1.symbols.gmt', 'MSigDB_C2_Canonical_Pathways' : 'c2.cp.v7.5.1.symbols.gmt', 'MSigDB_C3_microRNA_Targets': 'c3.mir.v7.5.1.symbols.gmt', 'MSigDB_C3_Transcription_Factor_Targets': 'c3.tft.v7.5.1.symbols.gmt', 'MSigDB_C4_Cancer_Gene_Neighborhoods': 'c4.cgn.v7.5.1.symbols.gmt', 'MSigDB_C4_Cancer_Modules': 'c4.cm.v7.5.1.symbols.gmt', 'MSigDB_C5_Gene_Ontology': 'c5.go.v7.5.1.symbols.gmt', 'MSigDB_C5_Human_Phenotype_Ontology': 'c5.hpo.v7.5.1.symbols.gmt', 'MSigDB_C6_Oncogenic_Signature': 'c6.all.v7.5.1.symbols.gmt', 'MSigDB_C7_ImmuneSigDB': 'c7.immunesigdb.v7.5.1.symbols.gmt', 'MSigDB_C7_Vaccine_Reponse': 'c7.vax.v7.5.1.symbols.gmt', 'MSigDB_C8_Cell_Type_Signature': 'c8.all.v7.5.1.symbols.gmt'},
    default = 'MSigDB_Hallmark',
    section = 'GS_Libraries_Section'
)%}

In [ ]:
%%appyter code_exec
{% set interactive_plot = BoolField(
    name = 'interactive_plot', 
    label = 'Interactive plots?', 
    default = 'true', 
    description = 'Check if User wants interactive plots', 
    section = 'Visualization_Section'
)%}

{% set visualization_method = ChoiceField(
    name = 'visualization_method', 
    label = 'Visualization Methods', 
    choices = {'PCA': 'PCA', 't-SNE': 't-SNE'},
    default = 'PCA', 
    description = 'Select a visualization method', 
    section = 'Visualization_Section'
)%}

{% set nr_genes = IntField(
    name = 'nr_genes', 
    label = 'Genes for Dimension Reduction', 
    min = 0, 
    max = 30000, 
    default = 2500, 
    description = 'The maximum number of genes for dimension reduction', 
    section = 'Visualization_Section'
)%}

{% set filter_genes = BoolField(
    name = 'filter_genes', 
    label = 'Filter genes?', 
    default = 'false',
    description = 'Check if User wants to filter genes with lowest variances', 
    section = 'Visualization_Section'
)%}

{% set low_expression_threshold = FloatField(
    name = 'low_expression_threshold', 
    label = 'Low expression threshold', 
    default = 0.3, 
    description = 'Threshold to filter out low expression genes. The value should vary based on the user dataset.', 
    section = 'Visualization_Section'
)%}

{% set logCPM_normalization = BoolField(
    name = 'logCPM_normalization', 
    label = 'logCPM normalization?', 
    default = 'false', 
    description = 'Check if User wants the dataset to be logCPM-transformed', 
    section = 'Visualization_Section'
)%}

{% set log_normalization = LogNormalizationField(
    name = 'log_normalization',
    label = 'log normalization?',
    urlhashkey = 'expression_unit',
    default = 'false', 
    description = 'Check if User wants the dataset to be log-transformed', 
    section = 'Visualization_Section'
)%}

{% set z_normalization = BoolField(
    name = 'z_normalization', 
    urlhashkey = 'expression_unit',
    label = 'Z normalization?', 
    default = 'true', 
    description = 'Check if User wants the dataset to be normalized with Z-normalized method', 
    section = 'Visualization_Section'
)%}

{% set q_normalization = BoolField(
    name = 'q_normalization', 
    label = 'Quantile normalization?', 
    default = 'false', 
    description = 'Check if User wants the dataset to be normalized with Quantile normalization method', 
    section = 'Visualization_Section'
)%}

{% set save_intermediate = BoolField(
    name = 'save_intermediate', 
    label = 'Save intermediate files?', 
    default = 'false', 
    description = 'Check if User wants to save intermediate analysis files', 
    section = 'Visualization_Section'
)%}

In [ ]:
%%appyter code_exec
{% set diff_gex_method = DiffgexChoiceField(
    name = 'DiffgexChoiceField',
    label = 'Differential expression analysis method',
    urlhashkey = 'expression_unit',
    choices = {'limma_voom': 'limma_voom', 'characteristic direction': 'characteristic_direction', 'edgeR': 'edgeR', 'DESeq2': 'DESeq2'},
    default = 'characteristic direction', 
    description = 'Set a method to get differentially expressed genes', 
    section = 'DEG_Section')
%}

{% set diff_gex_plot_method = ChoiceField(
    name = 'diff_gex_plot_method',
    label = 'Differential expression analysis plotting method',
    choices = {'Volcano plot': 'volcano','MA plot': 'MA_plot'},
    default = 'Volcano plot', 
    description = 'Set a plot method to see differentially expressed genes. Available for limma/edgeR/DESeq2.', 
    section = 'DEG_Section')
%}

{% set pvalue_threshold = FloatField(
    name = 'pvalue_threshold', 
    label = 'P-value threshold', 
    min = 0, 
    max = 1, 
    default = 0.05, 
    step = 0.05,
    description = 'Threshold to highlight significantly differentially expressed genes', 
    section = 'DEG_Section')
%}
{% set logfc_threshold = FloatField(
    name = 'logfc_threshold', 
    label = 'logFC threshold',
    min = 0,
    max = 1000,
    default = 1.5, 
    description = 'Threshold to highlight diffentially expressed genes', 
    section = 'DEG_Section')
%}

{% set gene_topk = IntField(
    name = 'gene_topk', 
    label = 'Maximum genes for Enrichr', 
    min = 0, 
    max = 1000, 
    default = 500, 
    description = 'The maximum number of genes discovered by the Characteristic Direction method', 
    section = 'DEG_Section')
%}

In [ ]:
%%appyter code_exec
{% set permutation = IntField(
    name = 'permutations', 
    label = 'Permutations', 
    min = 0, 
    max = 30000, 
    default = 2000, 
    description = 'The number of randomized permutations to estimate ES distributions', 
    section = 'blitzGSEA_Section')
%}

{% set min_genes = IntField(
    name = 'min_genes', 
    label = 'Minimum number of genes',
    min = 0, 
    max = 30000, 
    default = 5, 
    description = 'The minimum number of genes in geneset', 
    section = 'blitzGSEA_Section')
%}

{% set max_genes = IntField(
    name = 'max_genes', 
    label = 'Maximum number of genes',
    min = 0, 
    max = 500, 
    default = 500, 
    description = 'The maximum number of genes in geneset', 
    section = 'blitzGSEA_Section')
%}

{% set anchors = IntField(
    name = 'anchors', 
    label = 'Anchors',
    min = 0, 
    max = 30000, 
    default = 20, 
    description = 'The number of gene set size distributions calculated. Remaining are interpolated.', 
    section = 'blitzGSEA_Section')
%}

{% set symmetric = BoolField(
    name = 'symmetric', 
    label = 'Symmetric', 
    default = 'false', 
    description = 'Use same distribution parameters for negative and positive ES. If False estimate them separately.', 
    section = 'blitzGSEA_Section')
%}

{% set n_table = ChoiceField(
    name = 'n_table', 
    label = 'Number of gene set in full table and top table',
    choices = ['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50'],
    default = '10',
    description = 'The top n enriched gene sets displayed in top table', 
    section = 'blitzGSEA_Section')
%}

In [ ]:
%%appyter code_exec
# Parse Xena Browser post data
DE_params = r"""{{DE_params.value}}"""
try:
    J = json.loads(DE_params)
except: # empty form
    display(Markdown("<span style=\"background-color:#f2dede\">Error! No input.</span>"))
    sleep(10)
    raise SystemExit(1)
expr_dataset_name = J['preferredExpression']['name']
xena_host = J['preferredExpression']['host']
probemap_name = J['preferredExpression'].get('probemap')
unit = J['preferredExpression'].get('unit')
expr_data_filename = xenaFileDownloadLink(xena_host, expr_dataset_name)
probemap_filename = xenaFileDownloadLink(xena_host, probemap_name)
samples = J["samples"]
values = J["data"]["req"]["values"][0]
codes = J["data"]["codes"]

logPattern = re.compile("^log", flags=re.IGNORECASE)
patternPseudoCount = re.compile( "\+\s*[0-9]+(\.[0-9]+)?\s*\)")
patternNumber = re.compile("[0-9]+(\.[0-9]+)?")
logData = unit and re.search(logPattern, unit)
pseudocount = (unit and re.search(patternPseudoCount, unit) and float(re.search(patternNumber, re.search(patternPseudoCount, unit).group(0)).group(0))) or 0

assert(len(samples) == len(values))

In [ ]:
%%appyter code_exec
case_group = {{case_group.value}}
control_group = {{control_group.value}}

gs_library = "{{gs_library.value}}"

interactive_plot = {{interactive_plot.value}}
filter_genes = {{filter_genes.value}}
low_expression_threshold = {{low_expression_threshold.value}}
nr_genes = {{nr_genes.value}}
logCPM_normalization = {{logCPM_normalization}}
log_normalization = {{log_normalization}}
z_normalization = {{z_normalization}}
q_normalization = {{q_normalization}}
save_intermediate = {{save_intermediate}}

diff_gex_method = "{{diff_gex_method.value}}"
diff_gex_plot_method = "{{diff_gex_plot_method.value}}"
pvalue_threshold = {{pvalue_threshold.value}}
logfc_threshold = {{logfc_threshold.value}}
gene_topk = {{gene_topk.value}}

permutation = {{permutation.value}}
min_genes = {{min_genes.value}}
max_genes = {{max_genes.value}}
anchors = {{anchors.value}}
symmetric = {{symmetric.value}}
n_table = {{n_table.value}}

In [ ]:
warnings.filterwarnings('ignore')
random.seed(0)
pandas2ri.activate()
if interactive_plot == True:
    plot_type = 'interactive'
else:
    plot_type = 'static'
chart_studio.tools.set_credentials_file(username='mjjeon', api_key='v0rpMa6lhST28Sq7XqtM')
results = {}
table_counter = 1
figure_counter = 1

# Build Subgroup 1 vs. Subgroup 2

In [ ]:
meta_df = build_meta_df(samples, values, codes)
meta_id_column_name, meta_class_column_name = meta_df.columns
meta_subgroup_column_name = "subgroup"

# build case vs. control subgroups
error = check_subgroups (control_group, case_group)
if error:
    display(Markdown("<span style=\"background-color:#f2dede\">Error! {}.</span>".format(error)))
    sleep(10)
    raise SystemExit(1)

case_subgroup_name = '+'.join(case_group).replace("/","_")
case_subgroup_name = re.sub(r'\W+', '_', case_subgroup_name) # all nonalphanum change to _
control_subgroup_name = '+'.join(control_group)
control_subgroup_name = re.sub(r'\W+', '_', control_subgroup_name) # all nonalphanum change to _
meta_df[meta_subgroup_column_name] = np.where(np.isin(meta_df[meta_class_column_name], case_group), case_subgroup_name, np.where(np.isin(meta_df[meta_class_column_name], control_group), control_subgroup_name,''))
subgroups = [control_subgroup_name, case_subgroup_name] # CONTROL first, CASE second
meta_df = meta_df[meta_df[meta_subgroup_column_name]!= '']

table_counter = display_object(table_counter, "Sample size for each subgroup. The table displays the number of samples in each subgroup.", meta_df.groupby(meta_subgroup_column_name)[meta_subgroup_column_name].count().to_frame(), istable=True)

n_case = len(meta_df[meta_df[meta_subgroup_column_name] == case_subgroup_name])
n_control  = len(meta_df[meta_df[meta_subgroup_column_name] == control_subgroup_name])
if n_case < 2 or n_control < 2:
    display(Markdown("<span style=\"background-color:#f2dede\">Error! Each subgroup needs to have at least 2 samples with gene expression data. \"{}\" group has {} samples. \"{}\" group has {} samples.</span>".format(case_subgroup_name, n_case, control_subgroup_name, n_control)))
    print ("Error!")
    sleep(10)
    raise SystemExit(1)

In [ ]:
# set memory, runtime and sample limit for the analysis
website = False
def sample_exceeded(sample_size, max_sample):
    display(Markdown("<span style=\"background-color:#f2dede\">There are {} samples in the analysis, which has exceeded the {} sample limit on this website. You can use Xena to filter down the samples first.</span>".format(sample_size, max_sample)))
    print ("Error!")
    sleep(10)
    raise SystemExit(1)

def time_exceeded(signo, frame): 
    display(Markdown("**The analysis is terminated because it has exceeded the maximum time limit.**"))
    print ("Error!")
    sleep(10)
    raise SystemExit(1)
    
def set_resource_limit(max_time, max_mem):
    soft, hard = resource.getrlimit(resource.RLIMIT_CPU)
    resource.setrlimit(resource.RLIMIT_CPU, (max_time, hard))
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    resource.setrlimit(resource.RLIMIT_AS, (max_mem, hard))
    signal.signal(signal.SIGXCPU, time_exceeded) 

if os.path.exists("../../analysis.xenahubs.net"):
    website = True

if website:
    max_time = 60 * 20 # max run time is 20 min
    max_mem = 19.9 * 1000 * 1000 * 1000 # max memmory allocated 16GB
    max_sample = 2000 # max sample size 2000 
    set_resource_limit(max_time, max_mem)

    # total number of samples in subgroups
    if len(meta_df) > max_sample:
        sample_exceeded(len(meta_df), max_sample)

# Load Dataset

In [ ]:
# Downloading expr data file
try:
    expr_data_filename = find_remote_file(expr_data_filename)
except:
    print(f"Error! Gene expression file not found at '{expr_data_filename}'")
    sleep(10)
    raise SystemExit(1)

print("Downloading expression dataset ... Large dataset will take longer.")
buffer = smart_open.smart_open(expr_data_filename,'r')
expr_df = parse_xena_expr(buffer, meta_df[meta_id_column_name].to_list())
print("Download of expression dataset complete")

# display expr table
table_counter = display_object(table_counter, "Gene expression data. The table displays the input gene expression dataset. Rows represent genes, columns represent samples, and values show the gene expression level.", expr_df, istable = True)

# display meta data table
table_counter = display_object(table_counter, "Metadata. The table displays the metadata associated with the samples in the gene expression dataset. Rows represent samples, columns represent metadata categories and subgroups.", meta_df, istable = True)

In [ ]:
# Match samples between the metadata (subgroup column) and the expression datasets
try:
    check_df(meta_df, meta_id_column_name)
except:
    print(f"Error! Column '{meta_id_column_name}' is not in metadata")
try:
    check_df(meta_df, meta_class_column_name)
except:
    print(f"Error! Column '{meta_class_column_name}' is not in metadata")

meta_df = meta_df[meta_df[meta_id_column_name].isin(expr_df.columns)] #filter metadata
expr_df = expr_df.groupby(expr_df.columns, axis=1).mean() # average across duplicated samples, remove sample duplications
expr_df = expr_df.loc[:,meta_df[meta_id_column_name]] #reorder
expr_df = expr_df.groupby(expr_df.index).sum() # add all values from the same gene, remove gene duplications

try:
    assert(meta_df.shape[0]==expr_df.shape[1])
except:
    print("Unexpected! Sample mismatch detected")
    sleep(10)
    raise SystemExit(1)

n_case = len(meta_df[meta_df[meta_subgroup_column_name] == case_subgroup_name])
n_control  = len(meta_df[meta_df[meta_subgroup_column_name] == control_subgroup_name])
if n_case < 2 or n_control < 2:
    display(Markdown("<span style=\"background-color:#f2dede\">Error! Each subgroup needs to have at least 2 samples with gene expression data. \"{}\" group has {} samples with gene expression data. \"{}\" group has {} samples with gene expression data.</span>".format(case_subgroup_name, n_case, control_subgroup_name, n_control)))
    print ("Error!")
    sleep(10)
    raise SystemExit(1)

In [ ]:
# convert non-HUGO probe_level expr_df to hugo-gene-level expr_df
to_hugo = False
if probemap_filename:
    try:
        probemap_filename = find_remote_file(probemap_filename)
        probemap_buffer = smart_open.smart_open(probemap_filename,'r')
        probemap_df = pd.read_csv(probemap_buffer, index_col = 0, sep = "\t", compression = "infer", header = None, comment = "#")
        if check_probe_level(probemap_df) != "HUGO":
            # convert expr_df to hugo_gene_level expr_df
            expr_df = convert_to_hugo(expr_df, probemap_df)
            to_hugo = True
    except:
        print(f"Error! Probemap file not found at '{probemap_filename}'")

In [ ]:
dataset = dict()
current_dataset = 'rawdata'
dataset[current_dataset] = expr_df
dataset['dataset_metadata'] = meta_df

In [ ]:
%%appyter markdown
{% if filter_genes.value == True %}
Filter out low expressed genes 
{% endif %}

In [ ]:
%%appyter code_exec
{% if filter_genes.value == True %}
## Filter out non-expressed genes
expr_df = expr_df.loc[expr_df.sum(axis = 1) > 0, :]

## Filter out lowly expressed genes
mask_low_vals = (expr_df > low_expression_threshold).sum(axis = 1) > 2
expr_df = expr_df.loc[mask_low_vals, :]
current_dataset += '+filter_genes'
dataset[current_dataset] = expr_df
{% endif %}

In [ ]:
# display expression data table - hugo converted
if (to_hugo):
    table_counter = display_object(table_counter, "Gene expression data. The table displays the quantified gene expression dataset after converting input data to using hugo gene names. Rows represent genes, columns represent samples, and values show the gene expression level.", dataset[current_dataset], istable = True)
    # display(create_download_link(dataset[current_dataset], filename="gene_level_expr.csv"))

In [ ]:
%%appyter markdown
{% if logCPM_normalization.value == True or log_normalization.value == True or z_normalization.value == True or q_normalization.value == True %}
# Normalize gene expression data
Normalization methods ({% if logCPM_normalization.value %}count per million (CPM), {% endif %} {% if log_normalization.value %} log transformation, {% endif %} {% if z_normalization.value %} Z normalization, {% endif %} {% if q_normalization.value %}quantile normalization {% endif %}) will be applied to convert raw read counts into informative measures of gene expression and remove factors that affect the analysis.
{% endif %}

In [ ]:
%%appyter code_exec
dataset, normalization = normalize(dataset, current_dataset, {{logCPM_normalization}}, {{log_normalization}}, {{z_normalization}}, {{q_normalization}})

{% if logCPM_normalization.value == True or log_normalization.value == True or z_normalization.value == True or q_normalization.value == True %}
table_counter = display_object(table_counter, "Normalized data. The table displays the expression values after normalization.", dataset[normalization], istable = True)
{% if save_intermediate.value == True %}
display(create_download_link(dataset[normalization], filename = "normalized_data.csv"))
{% endif %}
{% endif %}

# QC Plot: Total gene expression analysis

Total gene expression analysis calculates and displays the total gene expression for each sample in the gene expression dataset, facilitating assessment of the overall quality of the data.

In [ ]:
meta_df['sum'] = expr_df.sum().tolist()
fig = px.bar(meta_df.sort_values(by = 'sum'), x = meta_id_column_name, y = 'sum')

if plot_type == 'interactive':
        plotly.offline.iplot(fig)
else:
    py.image.ishow(fig)
figure_counter = display_object(figure_counter, "The total gene expression for each sample. The figure contains an interactive bar chart which displays the total gene expression (sum) mapped to each sample in the dataset. Additional information for each sample is available by hovering over the bars.", istable=False)

# Visualize Samples

In [ ]:
%%appyter markdown
{% if visualization_method.value == "PCA" %}
Principal Component Analysis (PCA) (Clark et al. 2011) is a statistical technique used to identify global patterns in high-dimensional datasets. It is commonly used to explore the similarity of biological samples in gene expression datasets. To achieve this, gene expression values are transformed into Principal Components (PCs), a set of linearly uncorrelated features which represent the most relevant sources of variance in the data, and subsequently visualized using a scatter plot.
{% endif %}

In [ ]:
%%appyter code_exec
{% if visualization_method.value == "PCA" %}
method = "PCA"
{% endif %}
{% if visualization_method.value == "t-SNE" %}
method = "t-SNE"
{% endif %}

# Run analysis
results[method] = run_dimension_reduction(dataset = dataset, meta_id_column_name = meta_id_column_name, method = method,\
                         nr_genes = nr_genes, normalization = normalization, plot_type = plot_type)
# Display results
figure_counter = plot_samples(results[method], meta_id_column_name = meta_id_column_name, meta_class_column_name = meta_class_column_name, counter = figure_counter)


In [ ]:
%%appyter markdown
# {{diff_gex_method}} Differential Gene Expression

Gene expression signatures are alterations in the patterns of gene expression that occur as a result of cellular perturbations such as drug treatments, gene knock-downs or diseases. They can be quantified using differential gene expression (DGE) methods (Ritchie et al. 2015, Clark et al. 2014), which compare gene expression between two groups of samples to identify genes whose expression is significantly altered in the perturbation. 

In [ ]:
signatures = get_signatures(subgroups, dataset, normalization, diff_gex_method, meta_subgroup_column_name, meta_id_column_name, filter_genes, logData, pseudocount)

for label, signature in signatures.items():
    table_counter = display_object(table_counter, "Differentially expressed genes between {} using {}. The figure displays a browsable table containing the gene expression signature generated from a differential gene expression analysis. Every row of the table represents a gene; the columns display the estimated measures of differential expression.".format(label, diff_gex_method), signature, istable = True)
    display(create_download_link(signature, filename = "DEG_results_{}.csv".format(label)))

In [ ]:
%%appyter code_exec
{% if diff_gex_method.value == "limma" or diff_gex_method.value == "limma_voom" or diff_gex_method.value == "edgeR" or diff_gex_method.value == "DESeq2"%}
    
{% if diff_gex_plot_method.value == "volcano" %}
results['volcano_plot'] = {}
# Loop through signatures
for label, signature in signatures.items():
    results['volcano_plot'][label] = run_volcano(signature, label, dataset, pvalue_threshold, logfc_threshold, plot_type)
    plot_volcano(results['volcano_plot'][label])
    figure_counter = display_object(figure_counter, "Volcano plot for {}. The figure contains an interactive scatter plot which displays the log2-fold changes and statistical significance of each gene calculated by performing a differential gene expression analysis. Genes with logFC > {} and p-value < {} in red and genes with logFC < -{} and p-value < {} in blue. Additional information for each gene is available by hovering over it.".format(label, logfc_threshold, pvalue_threshold, logfc_threshold, pvalue_threshold), istable=False)

{% elif diff_gex_plot_method.value == "MA_plot" %}
# Initialize results
results['ma_plot'] = {}

# Loop through signatures
for label, signature in signatures.items():
    # Run analysis
    results['ma_plot'][label] = run_maplot(signature = signature, signature_label = label, pvalue_threshold = pvalue_threshold, logfc_threshold = logfc_threshold, plot_type = plot_type)
    # Display results
    plot_maplot(results['ma_plot'][label])
    figure_counter = display_object(figure_counter, "MA plot for {}. The figure contains an interactive scatter plot which displays the average expression and statistical significance of each gene calculated by performing differential gene expression analysis. Genes with logFC > {} and p-value < {} in red and genes with logFC < -{} and p-value < {} in blue. Additional information for each gene is available by hovering over it.".format(label, logfc_threshold, pvalue_threshold, logfc_threshold, pvalue_threshold), istable=False)

{% endif %}
{% endif %}

# blitzGSEA Full Table

[blitzGSEA](https://pubmed.ncbi.nlm.nih.gov/35143610/) is an algorithm developed by a group of researchers at Ma'ayan Labs. It has been evaluated to be significantly more performant than the [traditional GSEA](https://www.pnas.org/doi/10.1073/pnas.0506580102) implementation while producing similar results that are biologically consistent. The gene set libraries available to run blitzGSEA with through Xena is retrieved from the [Molecular Signatures Database (MSigDB)](http://www.gsea-msigdb.org/gsea/msigdb/collections.jsp). 

In [ ]:
HTML("<style>.dataframe td {white-space: nowrap;}</style>")
signature_df = reformat_signature(signatures, diff_gex_method)
signature_df.to_csv('signature.tsv', sep='\t', index=False)

n_file = open('n_value.tsv', 'w')
n_file.write(str(n_table))
n_file.close()

display(Markdown("### Gene Set Library: {}".format(gs_library)))
file = '/app/data/gsLibrary/' + gs_library
library = reformat_gmt(file)
full_table = run_blitzGSEA(signature_df, library)
full_table.reset_index(inplace=True) 
    
with pd.option_context('display.max_colwidth', 30):
    display(full_table.head(n_table))
table_counter = display_object(table_counter, "The table displays the enrichment score (es), normalized enrichment score (nes), p-value (pval), sidak correction (sidak), false discovery rate (fdr), gene set size (geneset_size), and leading genes (leading_edge) for each gene set. Each row is a gene set.", istable=True)
display(create_download_link(full_table, title="Download CSV file: {}", filename="blitzGSEA_full_table_{}.csv".format(gs_library)))

full_table.to_csv('full_table.tsv', sep='\t', index=None)
library = pd.read_csv(file)
library.to_csv('library.tsv', sep='\t', index=None)

# blitzGSEA Top Table, Running Sum Plot, and Detailed Output

Click below to access an interactive top table and view running sum plots and detailed tables of selected gene sets. 

In [ ]:
%%appyter markdown
**[View Gene Set Data Dashboard](../dashboard/{{_session}})**